In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [5]:
rs=121
ns=5
stocks=['aapl','googl','nvda','dal','xom','cvx','vz']

In [3]:
init_dir_path = str(os.getcwd())
data=pd.read_csv(init_dir_path+'/stock_hmm_output.csv',engine='python').drop(['Unnamed: 0'],axis=1)
features=['Return','Volatility','HighR','LowR']
target='State'

In [4]:
#With TimeSeries
#First Set Aside Testing data
final_test=[]
l=[]
ft_size=400
for x in data.groupby(['Ticker']):
    final_test.append(x[1][-ft_size:])
    l.append(x[1][:-ft_size])

In [ ]:
testing_size=500

tss=TimeSeriesSplit(n_splits=ns,test_size=testing_size)
training_list=[]
testing_list=[]
for i in range(ns):
    training_list.append([[],[]])
    testing_list.append([[],[]])
training_lists = {}
testing_lists = {}
for spread in range(2,25):
    for x in l:
        for (i,t) in enumerate(tss.split(x)):
            ((train_x,train_y),(test_x,test_y))=createSlidingWindow(x.iloc[t[0]],x.iloc[t[1]],features,target, window = spread)
            training_list[i][0] += train_x
            training_list[i][1] += train_y
            testing_list[i][0] += test_x
            testing_list[i][1] += test_y
    training_lists[spread] = training_list
    testing_lists[spread] = testing_list
    training_list=[]
    testing_list=[]
    for i in range(ns):
        training_list.append([[],[]])
        testing_list.append([[],[]])

In [6]:
with open('saved_training_lists.pkl', 'rb') as f:
    training_lists = pickle.load(f)

with open('saved_testing_lists.pkl', 'rb') as f:
    testing_lists = pickle.load(f)

In [7]:
def accuracies_model(model):
    accuracies = {}
    for window in training_lists.keys():
        training_list = training_lists[window]
        testing_list = testing_lists[window]
        a=0 
        for i in range(ns):
            model.fit(X=training_list[i][0],y=training_list[i][1])
            a+= accuracy_score(model.predict(testing_list[i][0]),testing_list[i][1])
        a=a/ns
        accuracies[window] = a
    return accuracies

In [16]:
model_names=['ada','decisiontree']
training_list  = training_lists[13]
testing_list = testing_lists[13]
accuracies=[]
models=[AdaBoostClassifier(n_estimators=200,random_state=121),DecisionTreeClassifier(random_state=rs)]
#Add Possible AdaBoostModels:
for m in models:
    a=0
    for i in range(ns):
        m.fit(X=training_list[i][0],y=training_list[i][1])
        a+= accuracy_score(m.predict(testing_list[i][0]),testing_list[i][1])
    a=a/ns
    accuracies.append(a)

In [17]:
accuracies

[0.873452625403344, 0.8026400704018772]

In [11]:
training_lists.keys()

dict_keys([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])

In [ ]:
model_names=['ada','decisiontree']
models=[AdaBoostClassifier(n_estimators=220,learning_rate=1,random_state=rs),DecisionTreeClassifier(random_state=rs)]
#Add Possible AdaBoostModels:
for ind, m in enumerate(models):
    acc_model = accuracies_model(m)
    plot_accuracies(acc_model, model_names[ind])

In [19]:
#13 seems best from above
window=13
model=AdaBoostClassifier()

parameters = {
    'n_estimators' : [ 180, 200,220],
    'learning_rate' : [1,10]
}
grid_search = GridSearchCV(model, parameters, cv = 5, n_jobs=-1)
training_list = training_lists[window]
testing_list = testing_lists[window]
a=[] 
for i in range(ns):
    grid_search.fit(X=training_list[i][0],y=training_list[i][1])
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)
    a.append(grid_search.cv_results_)

    #a+= accuracy_score(model.predict(testing_list[i][0]),testing_list[i][1])
#a=a/ns

AdaBoostClassifier(learning_rate=1, n_estimators=220)
0.8191994720131739
AdaBoostClassifier(learning_rate=1, n_estimators=200)
0.836738932314244
AdaBoostClassifier(learning_rate=1, n_estimators=180)
0.8448664639503141
AdaBoostClassifier(learning_rate=1, n_estimators=220)
0.850352636201106
AdaBoostClassifier(learning_rate=1, n_estimators=220)
0.8507606993751269
